# Преобразование категориальных признаков очищенных данных

## Импорт библиотек

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import re

## Открытие данных

In [3]:
df = pd.read_csv('../clean_data_without_outliers.csv', sep=';')

In [4]:
df.sample(3)

,number of rooms,area of apartment,number of floors,apartment floor,price,repair,bathroom,terrace,year of construction,elevator,type of house,parking,district,full address,concierge,garbage chute
61,3,53.0,9,2,3700000.0,дизайнерский,совмещенный,лоджия,1974,1.0,панельный,открытая во дворе,Индустриальный,"Пермский край, Пермь, Чердынская ул., 22",0,1
198,2,52.8,10,2,4500000.0,косметический,раздельный,лоджия,1992,1.0,кирпичный,открытая во дворе,Индустриальный,"Пермский край, Пермь, ул. Советской Армии, 49",0,0
1242,3,56.6,9,8,3000000.0,требует ремонта,раздельный,лоджия,1980,1.0,панельный,открытая во дворе,Кировский,"Пермский край, Пермь, Автозаводская ул., 27",0,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1702 entries, 0 to 1701
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   number of rooms       1702 non-null   int64  
 1   area of apartment     1702 non-null   float64
 2   number of floors      1702 non-null   int64  
 3   apartment floor       1702 non-null   int64  
 4   price                 1702 non-null   float64
 5   repair                1702 non-null   object 
 6   bathroom              1702 non-null   object 
 7   terrace               1702 non-null   object 
 8   year of construction  1702 non-null   int64  
 9   elevator              1702 non-null   float64
 10  type of house         1702 non-null   object 
 11  parking               1457 non-null   object 
 12  district              1702 non-null   object 
 13  full address          1702 non-null   object 
 14  concierge             1702 non-null   int64  
 15  garbage chute        

## Обнаружение категориальных признаков

In [6]:
cat_features = []
for column in df.columns:
    if str(df[column].dtype) == "object":
        cat_features.append(column)
print("Количество признаков: " + str(len(cat_features)))

Количество признаков: 7


In [7]:
cat_features

['repair',
 'bathroom',
 'terrace',
 'type of house',
 'parking',
 'district',
 'full address']

Все признаки кроме полного адреса мы закодируем с помощью метода one hot encoding

### Обработка ремонта, типа ванной, типа балкона, типа стен дома, района

In [8]:
simple_cat_features = ['repair', 'bathroom', 'terrace', 'type of house', 'district']

In [9]:
for cat_feature in simple_cat_features:
    print(cat_feature + str(df[cat_feature].unique()))

repair['косметический' 'евро' 'требует ремонта' 'дизайнерский']
bathroom['раздельный' 'совмещенный' 'несколько']
terrace['балкон' 'нет' 'лоджия']
type of house['панельный' 'кирпичный' 'монолитный' 'блочный' 'деревянный']
district['Индустриальный' 'Свердловский' 'Мотовилихинский' 'Кировский'
 'Орджоникидзевский' 'Дзержинский' 'Ленинский']


Данные признаки содержат только одно значение, поэтому их можно обрабатывать единым образом

In [10]:
from sklearn.preprocessing import OneHotEncoder

def get_one_hot_df(df, column):
    encoder = OneHotEncoder()
    new_df = pd.DataFrame(encoder.fit_transform(df[[column]]).toarray())
    new_df.columns = [column + '_' + feature[3:] for feature in encoder.get_feature_names()]
    return new_df

In [11]:
for cat_feature in simple_cat_features:
    one_hot_df = get_one_hot_df(df, cat_feature)
    df = pd.merge(df, one_hot_df, left_index=True, right_index=True)
    df = df.drop(cat_feature, axis=1)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1702 entries, 0 to 1701
Data columns (total 33 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   number of rooms             1702 non-null   int64  
 1   area of apartment           1702 non-null   float64
 2   number of floors            1702 non-null   int64  
 3   apartment floor             1702 non-null   int64  
 4   price                       1702 non-null   float64
 5   year of construction        1702 non-null   int64  
 6   elevator                    1702 non-null   float64
 7   parking                     1457 non-null   object 
 8   full address                1702 non-null   object 
 9   concierge                   1702 non-null   int64  
 10  garbage chute               1702 non-null   int64  
 11  repair_дизайнерский         1702 non-null   float64
 12  repair_евро                 1702 non-null   float64
 13  repair_косметический        1702 

### Кодирование типа парковки

In [13]:
df.loc[df.parking.isna(), 'parking'] = ""

In [14]:
unique_parking = set()
for parking_set in df.parking.unique():
    for parking_type in parking_set.split(','):
        unique_parking.add(parking_type.strip())
unique_parking.remove('')

In [15]:
unique_parking

{'за шлагбаумом во дворе',
 'наземная многоуровневая',
 'открытая во дворе',
 'подземная'}

In [16]:
def get_parking_type(string, parking_type):
    return float(parking_type in string)

In [17]:
for parking_type in unique_parking:
    df["parking_" + parking_type] = get_parking_type(df.parking, parking_type)
df = df.drop('parking', axis=1)

## Итоги борьбы с выбросами

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1702 entries, 0 to 1701
Data columns (total 36 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   number of rooms                  1702 non-null   int64  
 1   area of apartment                1702 non-null   float64
 2   number of floors                 1702 non-null   int64  
 3   apartment floor                  1702 non-null   int64  
 4   price                            1702 non-null   float64
 5   year of construction             1702 non-null   int64  
 6   elevator                         1702 non-null   float64
 7   full address                     1702 non-null   object 
 8   concierge                        1702 non-null   int64  
 9   garbage chute                    1702 non-null   int64  
 10  repair_дизайнерский              1702 non-null   float64
 11  repair_евро                      1702 non-null   float64
 12  repair_косметический

In [19]:
df.to_csv('../clean_categorizated_data_without_outliers.csv', sep=';', index=False)

__В ходе третьей обработки были:__
1. трансформированы категориальные признаки по методу one hot encoding